# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gmaps_key

### Functions

In [2]:
# Return the most recent date from the the list provided
def getMostRecentDir(dir_list):
    
    dates = [int(date) for date in map(formatDate,dir_list)]
    most_rec = str(max(dates))
    return formatDate(most_rec)

In [3]:
# Flips the format of the date from MMDDYYYY to YYYYMMDD and vice versa
def formatDate(date):
    return date[4:] + date[:4]

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Get most recent directory in output
output_path = os.path.join("..","WeatherPy","output")
output_dir_list = os.listdir(output_path)
recent_dir = getMostRecentDir(output_dir_list)

# Access the most recent directory and store the cities.csv file
weather_csv_path = os.path.join(output_path,recent_dir,"cities.csv")
weather_data = pd.read_csv(weather_csv_path)
weather_df = pd.DataFrame(weather_data)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sungai Penuh,-2.08,101.38,67.80,90,91,3.24,ID,1609773157
1,Rome,43.21,-75.46,33.80,86,90,1.83,US,1609773102
2,Cairns,-16.92,145.77,78.80,94,75,3.36,AU,1609773033
3,Kapaa,22.08,-159.32,73.40,78,75,10.29,US,1609773161
4,Rikitea,-23.12,-134.97,77.14,81,17,15.55,PF,1609773162


### Humidity Heatmap

In [6]:
# Configure gmaps
gmaps.configure(api_key = gmaps_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"]

In [37]:
# Add Heatmap layer to map
fig = gmaps.figure(layout={"height":"600px","width":"900px"},center=(30.0, 0.0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='600px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
